### EDA and data cleanup

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
"""Extremely important to keep both parameters -- otherwise it stops working"""
df = pd.read_csv("data/data.csv", encoding='cp1251', sep=";", thousands=' ', decimal=',') ; df

,Дата,ДатаДоставки,НомерЗаказаНаСайте,НовыйСтатус,СуммаЗаказаНаСайте,СуммаДокумента,МетодДоставки,ФормаОплаты,Регион,Группа2,...,СуммаДоставки,НомерСтроки,КоличествоПроданоКлиенту,ДатаЗаказаНаСайте,Телефон_new,ЭлектроннаяПочта_new,Клиент,ID_SKU,ГородМагазина,МагазинЗаказа
0,01.03.2017 11:41,07.03.2017 0:00,3998972_TR,Частичный возврат,12098,12247.0,Курьерская,Наличная,Москва,КРУПНОГАБАРИТНЫЙ ТОВАР,...,0,8,1,28.02.2017 0:00,55574854-48574951555577,106117_ju18@yandex.ru,Юлия,ID9010020114553,0,NaN
1,01.03.2017 12:22,15.03.2017 0:00,3999117_TR,Доставлен,4819,3759.0,Магазины,Безналичная,Челябинск,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,0,3,1,01.03.2017 0:00,55575453-56535648535679,107117_ku16@list.ru,Гость,IDL00028974351,0,NaN
2,01.03.2017 12:31,17.03.2017 0:00,3999122_TR,Доставлен,3325,3325.0,Магазины,Безналичная,Нижневартовск,ИГРУШКИ,...,0,1,1,01.03.2017 0:00,55574950-57515657535772,109117_mu17@mail.ru,Гость,IDL00038573351,0,NaN
3,01.03.2017 3:06,02.03.2017 0:00,3999363_TR,К отгрузке,4998,4998.0,Курьерская,Наличная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,0,4,0,01.03.2017 0:00,55574851-55545249535475,115117_su13@mail.ru,Гость,IDL00034303856,0,NaN
4,01.03.2017 11:38,14.03.2017 0:00,3999411_TR,Доставлен,528,528.0,Магазины,Безналичная,Ижевск,ИГРУШКИ,...,0,3,1,01.03.2017 0:00,55574854-56495552515179,guest_103117_gu28@workmail.ru,Гость,IDL00035791553,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691534,17.04.2017 9:54,24.05.2017 0:00,4411156_TR,Принят в ПВЗ,3240,3240.0,Магазины,Безналичная,Чита,КРУПНОГАБАРИТНЫЙ ТОВАР,...,0,1,1,17.04.2017 0:00,55574952-51535652495475,guest_103117_gu28@workmail.ru,Гость,IDL00040855553,0,NaN
691535,17.04.2017 10:10,19.04.2017 0:00,4411268_TR,Доставлен,1377,1377.0,Магазины,Безналичная,Москва,ИГРУШКИ,...,49,2,1,17.04.2017 0:00,55575054-53545152544971,guest_103117_gu28@workmail.ru,Гость,IDL00013484149,0,NaN
691536,17.04.2017 11:03,27.04.2017 0:00,4411571_TR,Доставлен,1098,2086.0,Магазины,Безналичная,Нижний Новгород,КОСМЕТИКА/ГИГИЕНА,...,0,1,1,17.04.2017 0:00,55575448-49564957485078,97117_au19@mail.ru,Анна,IDL00026050250,0,NaN
691537,17.04.2017 11:03,27.04.2017 0:00,4411571_TR,Доставлен,1098,2086.0,Магазины,Безналичная,Нижний Новгород,КОСМЕТИКА/ГИГИЕНА,...,0,2,1,17.04.2017 0:00,55575448-49564957485078,97117_au19@mail.ru,Анна,IDL00026050654,0,NaN


In [3]:
df1 = df.replace('..', np.nan)
total = df1.isnull().sum().sort_values(ascending=False)
percent = (df1.isnull().sum() / df1.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.reset_index(inplace=True)
missing_data.rename(columns={'index': 'Attribute'}, inplace=True)
missing_data.head(15)

# Code snippet provided by Mr. Argirov during 1st year of DSBA, still used by me

,Attribute,Total,Percent
0,МагазинЗаказа,686556,0.992794
1,ПричинаОтмены,624836,0.903544
2,ПВЗ_код,228886,0.330981
3,Группа4,105625,0.152739
4,Маржа,76913,0.111220
5,ЦенаЗакупки,76913,0.111220
6,ТипТовара,74690,0.108005
7,Группа2,74686,0.108000
8,Группа3,74686,0.108000
9,ЭлектроннаяПочта_new,13117,0.018968


#### Transforming columns composition, data types and names

In [4]:
def transform_date(date_string: str):
    format_string = "%d.%m.%Y %H:%M"
    try:
        new_date = datetime.strptime(date_string, format_string)
        return new_date
    except:
        return datetime(1970, 1, 1, 0, 0, 0)


def transform_status(status_string: str, cancel_reason: str, is_cancelled: int) -> str:
    if pd.isna(cancel_reason) == False:
        return 'Cancelled'
    if is_cancelled == 1:
        return 'Cancelled'
    if cancel_reason == 'Возврат товара':
        return 'Returned'
    if status_string == 'Возврат':
        return 'Returned'
    if status_string == 'Доставлен':
        return 'Delivered'
    if status_string == 'Отменен':
        return 'Cancelled'
    if status_string == 'В процессе':
        return 'In Progress'
    else:
        return 'N/A'
      

def transform_new_status(status_string: str, is_cancelled: int) -> str:
    if is_cancelled == 1:
        return 'Cancelled'
    if status_string == 'Доставлен':
        return 'Delivered'
    if status_string in ['Возврат', 'Частичный возврат', 'Возврат из ПВЗ']:
        return 'Returned'
    if status_string in ['Отменен', 'Отменяется']:
        return 'Cancelled'
    else:
        return 'In Progress'


df['Order_Date'] = df['Дата'].apply(lambda x: transform_date(x))
df['Delivery_Date'] = df['ДатаДоставки'].apply(lambda x: transform_date(x))           #just date w/o time
df['Payment_Type'] = [1 if i == 'Наличная' else 0 for i in df['ФормаОплаты']]
df['Is_Cancelled'] = [1 if i == 'Да' else 0 for i in df['Отменено']]
df['Is_Moscow'] = [1 if i == 'Москва' else 2 if i == 'МО' else 0 for i in df['Гео']]

df.drop(columns=['МесяцДатыЗаказа', 'ГодДатыЗаказа', 'ЭлектроннаяПочта_new',
                'ГородМагазина', 'ID_SKU', 'МагазинЗаказа', 'Дата', 'ДатаДоставки',
                'ФормаОплаты', 'Отменено', 'НомерСтроки', 'ПВЗ_код', 'Клиент', 'ДатаЗаказаНаСайте', 'Гео'], axis=1, inplace=True)


df['Status'] = df[['Статус', 'ПричинаОтмены', 'Is_Cancelled']].apply(lambda x: transform_status(x['Статус'], x['ПричинаОтмены'], x['Is_Cancelled']), axis=1)
df['New_Status'] = df[['НовыйСтатус', 'Is_Cancelled']].apply(lambda x: transform_new_status(x['НовыйСтатус'], x['Is_Cancelled']), axis=1)

df.drop(columns=['ПричинаОтмены', 'Is_Cancelled', 'Статус', 'НовыйСтатус'], axis=1, inplace=True)

df.rename(columns={'НомерЗаказаНаСайте': 'Order_ID', 'СуммаЗаказаНаСайте': 'Initial_Order_Sum', 'СуммаДокумента': 'Final_Order_Sum',
                   'МетодДоставки': 'Delivery_Type', 'Регион': 'Region', 'Группа2': 'Item_Group', 'Группа3': 'Item_Group1', 'Группа4': 'Item_Group2',
                   'Тип': 'Item_Type', 'ТипТовара': 'Item_Type1', 'Номенклатура': 'Item_Name', 'Количество': 'Amount', 'Цена': 'Price', 'СуммаСтроки': 'Items_Total_Sum',
                   'ЦенаЗакупки': 'Item_Cost', 'Маржа': 'Total_Profit_Margin', 'СуммаУслуг': 'Service_Cost', 'СуммаДоставки': 'Delivery_Cost', 'КоличествоПроданоКлиенту': 'Amount_Sold',
                   'Телефон_new': 'Client_ID'}, inplace=True)

In [5]:
df2 = df.replace('..', np.nan)
total1 = df2.isnull().sum().sort_values(ascending=False)
percent1 = (df2.isnull().sum() / df2.isnull().count()).sort_values(ascending=False)
missing_data1 = pd.concat([total1, percent1], axis=1, keys=['Total', 'Percent'])
missing_data1.reset_index(inplace=True)
missing_data1.rename(columns={'index': 'Attribute'}, inplace=True)
missing_data1.head(9)

,Attribute,Total,Percent
0,Item_Group2,105625,0.152739
1,Item_Cost,76913,0.111220
2,Total_Profit_Margin,76913,0.111220
3,Item_Type1,74690,0.108005
4,Item_Group,74686,0.108000
5,Item_Group1,74686,0.108000
6,Region,5142,0.007436
7,Order_ID,4,0.000006
8,Delivery_Cost,0,0.000000


#### Reordering the columns for more practicality, excluding the last level of hierarchy

In [6]:
df = df[['Client_ID', 'Order_ID', 'Order_Date', 'Delivery_Date', 'Status', 'New_Status', 'Initial_Order_Sum', 'Amount', 'Final_Order_Sum', 'Amount_Sold', 'Price',
               'Item_Cost', 'Items_Total_Sum', 'Total_Profit_Margin', 'Service_Cost', 'Delivery_Cost', 'Payment_Type', 'Delivery_Type', 'Item_Type', 'Item_Type1', 'Item_Name',
               'Item_Group', 'Item_Group1', 'Region', 'Is_Moscow']] # exclude Item_Group2

df.dropna(subset=['Item_Group', 'Item_Group1'], inplace=True)

C:\Users\ALIENWARE\AppData\Local\Temp\ipykernel_8952\591900026.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['Item_Group', 'Item_Group1'], inplace=True)


In [7]:
df.to_csv('data/data_clean.csv', encoding='utf-8', sep=';')

In [8]:
df[:10000].to_csv('data/data_clean_snippet.csv', encoding='utf-8', sep=';')

### Data Overview

In [9]:
import calendar

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#### Creating new fields

In [10]:
df["is_delivered"] = df["New_Status"].isin(["Delivered", "In Progress"])
df["is_cancel_or_return"] = df["New_Status"].isin(["Cancelled", "Returned"])

# Monetary values only for delivered orders
df["Revenue"] = np.where(df["is_delivered"], df["Final_Order_Sum"], 0.0)
df["Margin"] = np.where(df["is_delivered"], df["Total_Profit_Margin"], 0.0)
df["Month"]       = df["Order_Date"].dt.to_period("M")
df["Month_str"]   = df["Month"].dt.strftime("%B")
df["Weekday"]     = df["Order_Date"].dt.weekday
df["Weekday_str"] = df["Order_Date"].dt.day_name()
df["Hour"]        = df["Order_Date"].dt.hour

C:\Users\ALIENWARE\AppData\Local\Temp\ipykernel_8952\4178424522.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_delivered"] = df["New_Status"].isin(["Delivered", "In Progress"])
C:\Users\ALIENWARE\AppData\Local\Temp\ipykernel_8952\4178424522.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_cancel_or_return"] = df["New_Status"].isin(["Cancelled", "Returned"])
C:\Users\ALIENWARE\AppData\Local\Temp\ipykernel_8952\4178424522.py:5: SettingWithCopyWarning: 
A value is trying to be set on a co

#### Styler Cell

In [25]:
def kpi_agg(gr):
    out = pd.Series({
        "Revenue"              : gr["Revenue"].sum(),
        "Margin"               : gr["Margin"].sum(),
        "Total items"          : gr["Amount"].sum(),
        "Unique orders"        : gr["Order_ID"].nunique(),
        "Unique clients"       : gr["Client_ID"].nunique(),
        "Avg ticket (₽)"       : gr.loc[gr["is_delivered"], "Final_Order_Sum"].mean(),
        "Avg margin / item"    : (gr["Margin"].sum() / gr["Amount"].sum()) if gr["Amount"].sum() else 0,
        "Margin %"             : (gr["Margin"].sum() / gr["Revenue"].sum()) if gr["Revenue"].sum() else 0,
        "Avg qty / order"      : gr["Amount"].sum() / gr["Order_ID"].nunique()})
    return out


def style_tbl(tbl):
    return (tbl.style
            .background_gradient(cmap="Blues", subset=["Revenue", "Margin"])
            .background_gradient(cmap="Blues", subset=["Margin %"])
            .background_gradient(cmap="Blues", subset=["Avg ticket (₽)"])
            .format({
                "Revenue": "{:,.0f} ₽", "Margin": "{:,.0f} ₽",
                "Avg ticket (₽)": "{:,.0f} ₽",
                "Avg margin / item": "{:,.1f} ₽",
                "Margin %": "{:.1%}", "Total items": "{:.0f}",
                "Unique orders": "{:.0f}", "Unique clients": "{:.0f}", "Avg qty / order": "{:.2f}"}))

#### Buy-out and overall sales data

In [26]:
monthly_kpi = df.groupby("Month_str").apply(kpi_agg).sort_index(key=lambda s:pd.to_datetime(s, format="%B").month).round(2)

# Placed vs Purchased vs Buy-out
monthly_buyout = (df.groupby("Month_str").agg(placed_items  = ("Amount", "sum"), purchased_items = ("Amount_Sold", "sum")))
monthly_buyout["Buy-out %"] = 100 * monthly_buyout["purchased_items"] / monthly_buyout["placed_items"]

display(style_tbl(monthly_kpi))
display(monthly_buyout.style.format("{:,.0f}").background_gradient(cmap="Blues"))

C:\Users\ALIENWARE\AppData\Local\Temp\ipykernel_8952\1142780043.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monthly_kpi = df.groupby("Month_str").apply(kpi_agg).sort_index(key=lambda s:pd.to_datetime(s, format="%B").month).round(2)


,Revenue,Margin,Total items,Unique orders,Unique clients,Avg ticket (₽),Avg margin / item,Margin %,Avg qty / order
Month_str,,,,,,,,,
March,"905,019,182 ₽","45,863,499 ₽",463357,91431,66560,"4,081 ₽",99.0 ₽,5.0%,5.07
April,"872,299,474 ₽","43,866,078 ₽",464049,86987,63348,"4,048 ₽",94.5 ₽,5.0%,5.33


,placed_items,purchased_items,Buy-out %
Month_str,,,
April,"464,049","377,629",81
March,"463,357","382,137",82


In [27]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(
    x=monthly_buyout.index, y=monthly_buyout["placed_items"],
    name="Placed items"
))
fig.add_trace(go.Bar(
    x=monthly_buyout.index, y=monthly_buyout["purchased_items"],
    name="Purchased items"
))
fig.add_trace(go.Scatter(
    x=monthly_buyout.index, y=monthly_buyout["Buy-out %"],
    mode="lines+markers", name="Buy-out %", yaxis="y2"
), secondary_y=True)

fig.update_layout(
    title="Placed vs Purchased items and Buy-out %",
    yaxis_title="Items", yaxis2_title="Buy-out %",
    barmode="group"
)
fig.show()

#### Daily Sales Data

In [28]:
weekday_kpi = (df.groupby("Weekday_str").apply(kpi_agg).reindex(list(calendar.day_name)).round(2))
display(style_tbl(weekday_kpi))

C:\Users\ALIENWARE\AppData\Local\Temp\ipykernel_8952\2239606899.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,Revenue,Margin,Total items,Unique orders,Unique clients,Avg ticket (₽),Avg margin / item,Margin %,Avg qty / order
Weekday_str,,,,,,,,,
Monday,"269,144,957 ₽","12,245,447 ₽",144852,25696,21613,"4,131 ₽",84.5 ₽,5.0%,5.64
Tuesday,"279,944,439 ₽","12,853,132 ₽",146798,27323,23020,"3,985 ₽",87.6 ₽,5.0%,5.37
Wednesday,"321,963,524 ₽","14,355,197 ₽",163147,30804,25669,"3,988 ₽",88.0 ₽,4.0%,5.30
Thursday,"289,886,184 ₽","13,803,684 ₽",150531,28636,24114,"4,017 ₽",91.7 ₽,5.0%,5.26
Friday,"235,486,048 ₽","12,310,984 ₽",125460,24786,21123,"4,019 ₽",98.1 ₽,5.0%,5.06
Saturday,"177,390,340 ₽","11,145,914 ₽",91548,19470,16728,"4,226 ₽",121.8 ₽,6.0%,4.70
Sunday,"203,503,164 ₽","13,015,218 ₽",105070,21703,18767,"4,201 ₽",123.9 ₽,6.0%,4.84


In [29]:
pivot_rev = df.pivot_table(
    index="Weekday_str", columns="Hour", values="Revenue", aggfunc="sum"
).reindex(list(calendar.day_name))

pivot_margin = df.pivot_table(
    index="Weekday_str", columns="Hour", values="Margin", aggfunc="sum"
).reindex(list(calendar.day_name))

fig1 = px.imshow(pivot_rev, labels=dict(x="Hour", y="", color="Revenue"),
                 title="Revenue heat-map (weekday × hour)")
fig1.update_yaxes(autorange="reversed")
fig1.show()

fig2 = px.imshow(pivot_margin, labels=dict(x="Hour", y="", color="Margin"),
                 title="Margin heat-map (weekday × hour)")
fig2.update_yaxes(autorange="reversed")
fig2.show()

In [30]:
hourly = (
    df.groupby("Hour")[["Revenue", "Margin"]]
      .sum()
      .sort_index()
)

fig = make_subplots()
fig.add_trace(go.Scatter(x=hourly.index, y=hourly["Revenue"],
                         mode="lines+markers", name="Revenue"))
fig.add_trace(go.Scatter(x=hourly.index, y=hourly["Margin"],
                         mode="lines+markers", name="Margin"))
fig.update_layout(title="Revenue & Margin by Hour", xaxis_title="Hour")
fig.show()

In [32]:
hourly_stats = (df[df["is_delivered"]].groupby("Hour").agg(avg_ticket=("Final_Order_Sum", "mean"), orders_cnt=("Order_ID", "nunique")).reset_index())

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=hourly_stats["Hour"],
        y=hourly_stats["avg_ticket"],
        mode="lines+markers",
        name="Average ticket (₽)",
        line=dict(color="#1f77b4")          # blue
    )
)

fig.add_trace(
    go.Scatter(
        x=hourly_stats["Hour"],
        y=hourly_stats["orders_cnt"],
        mode="lines+markers",
        name="Orders count",
        line=dict(color="#e749dc"),         # orange
        yaxis="y2"                          # ← secondary axis
    )
)

fig.update_layout(
    title="Average Ticket and Number of Orders by Hour",
    xaxis_title="Hour of day",
    yaxis=dict(title="Average ticket, ₽"),
    yaxis2=dict(title="Orders count",
                overlaying="y", side="right"),  # twin axis
    legend=dict(yanchor="top", y=0.99, x=0.01)
)

fig.show()